In [ ]:
!pip install datasets

# Test execution based eval

In [ ]:
from datasets import load_dataset

ds = load_dataset("neulab/odex", "en", split="test")

In [ ]:
import json

with open('fid.cmd_dev.codet5.t10.json', 'r') as f:
    dev_data = json.load(f)

with open('fid.cmd_test.codet5.t10.json', 'r') as f:
    test_data = json.load(f)

with open('fid.cmd_train.codet5.t10.json', 'r') as f:
    train_data = json.load(f)

with open('fid.cmd_tmp.codet5.t10.json', 'r') as f:
    temp_data = json.load(f)

In [ ]:
dev_ids = []
for data in dev_data:
    dev_ids.append(data['id'])


test_ids = []
for data in test_data:
    test_ids.append(data['id'])


train_ids = []
for data in train_data:
    train_ids.append(data['id'])

tmp_ids = []
for data in temp_data:
    tmp_ids.append(data['id'])

In [ ]:
dev_ids1 = [item.split('-')[0] for item in dev_ids]
test_ids1 = [item.split('-')[0] for item in test_ids]
train_ids1 = [item.split('-')[0] for item in train_ids]
tmp_ids1 = [item.split('-')[0] for item in tmp_ids]

In [ ]:
metric_ids = list(ds['task_id'])
metric_ids = [str(num) for num in metric_ids]
type(metric_ids[0])

str

In [ ]:
common_ids = list(set(metric_ids) & set(test_ids1))

In [ ]:
len(set(metric_ids))

333

In [ ]:
final_list = []
for data in ds:
    if str(data['task_id']) in common_ids:
        temp_dict = dict()
        tempd = dict()
        tempd['test_start'] = data['test_start']
        tempd['entry_point'] = data['entry_point']
        tempd['suffix'] = data['suffix']
        tempd['test'] = data['test']
        tempd['prompt'] = data['prompt']
        temp_dict[str(data['task_id'])] = tempd
        final_list.append(temp_dict)

In [ ]:
len(final_list)

106

In [ ]:
import json

# Write the list into a JSON file
with open('unittests1.json', 'w') as file:
    # Use json.dump to write the list to the file
    json.dump(final_list, file, indent=2)

# Create a json file for all the question_ids and unit tests

In [ ]:
!pip install evaluate

In [ ]:
import json
import evaluate
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

def pass_at_k(result_file, unittest_file):
    with open(unittest_file, 'r') as f:
        unittests = json.load(f)

    main_ut = {}

    for item in unittests:
        for key, value in item.items():
            main_ut[key] = value

    # unittests is without dash
    # results file with dash

    # select the examples which have unit test
    selected_predictions = []
    with open(result_file, 'r') as f:
        for line in f:
            pred = json.loads(line)
            qsn = pred["question_id"].split("-")[0]
            if qsn in main_ut:
                selected_predictions.append(pred)
    print(f"selected {len(selected_predictions)} examples with unit test")

    # run the test
    # load the metric from huggingface
    code_eval_metric = evaluate.load("code_eval")
    preds = []
    tests = []
    for prediction in selected_predictions:
        suffix = ""
        question_id = prediction["question_id"]
        qsn = question_id.split("-")[0]
        unittest = main_ut[qsn]
        entry_point = unittest["entry_point"]
        # test_func = f"\n{unittest['test']}\ncheck({entry_point})"
        test_func = f"\n{unittest['test']}"

        # wrap the generated code to a runnable function
        if isinstance(prediction['clean_code'], list):
            runnable_func = [f"{unittest['prompt']}{x}{suffix}" for x in prediction['clean_code']]
        else:
            runnable_func = [f"{unittest['prompt']}{prediction['clean_code']}{suffix}"]

        preds.append(runnable_func)
        tests.append(test_func.replace('candidate', unittest['entry_point']))

        # tests.append(test_func)

    r = code_eval_metric.compute(
        predictions=preds,
        references=tests,
        k=[1, 5, 10, 50, 100, 150, 200],
        num_workers=8,
    )

    print(r[0])
    return r

result_file = 'test_results_test_same.json'
unittest_file = 'unittests1.json'
r = pass_at_k(result_file, unittest_file)


selected 110 examples with unit test
["def f_3283984():\n\treturn print('4a4b4c'.decode('hex'))"]

['\n    assert candidate() == "JKL"\n']
["def f_14971373():\n\treturn subprocess.call(['ps -ef | grep something | wc -l'])"]

["\n    output = b'  PID TTY          TIME CMD\\n  226 pts/1    00:00:00 bash\\n  285 pts/1    00:00:00 python3\\n  352 pts/1    00:00:00 ps\\n'\n    subprocess.check_output = Mock(return_value = output)\n    assert candidate() == output\n"]
['def f_18079563(s1, s2):\n\treturn np.intersection(s1, s2, ignore_index=True)']

['\n    x1, x2 = pd.Series([1,2]), pd.Series([1,3])\n    assert candidate(x1, x2).equals(pd.Series([1]))\n', '\n    x1, x2 = pd.Series([1,2]), pd.Series([1,3, 10, 4, 5, 9])\n    assert candidate(x1, x2).equals(pd.Series([1]))\n', '\n    x1, x2 = pd.Series([1,2]), pd.Series([1,2, 10])\n    assert candidate(x1, x2).equals(pd.Series([1, 2]))\n']
["def f_26153795(when):\n\treturn datetime.datetime.strptime(when, '%Y-%m-%dT%H:%M:%SZ')"]

["\n    assert

In [ ]:
r[1]

defaultdict(list,
            {1: [(0,
               {'task_id': 1,
                'passed': True,
                'result': 'passed',
                'completion_id': 0})],
             2: [(0,
               {'task_id': 2,
                'passed': True,
                'result': 'passed',
                'completion_id': 0})],
             3: [(0,
               {'task_id': 3,
                'passed': True,
                'result': 'passed',
                'completion_id': 0})],
             0: [(0,
               {'task_id': 0,
                'passed': True,
                'result': 'passed',
                'completion_id': 0})],
             4: [(0,
               {'task_id': 4,
                'passed': True,
                'result': 'passed',
                'completion_id': 0})],
             6: [(0,
               {'task_id': 6,
                'passed': True,
                'result': 'passed',
                'completion_id': 0})],
             5: [(0,
             

In [ ]:
print(r)

({'pass@1': 0.7636363636363637}, defaultdict(<class 'list'>, {1: [(0, {'task_id': 1, 'passed': True, 'result': 'passed', 'completion_id': 0})], 2: [(0, {'task_id': 2, 'passed': True, 'result': 'passed', 'completion_id': 0})], 3: [(0, {'task_id': 3, 'passed': True, 'result': 'passed', 'completion_id': 0})], 0: [(0, {'task_id': 0, 'passed': True, 'result': 'passed', 'completion_id': 0})], 4: [(0, {'task_id': 4, 'passed': True, 'result': 'passed', 'completion_id': 0})], 6: [(0, {'task_id': 6, 'passed': True, 'result': 'passed', 'completion_id': 0})], 5: [(0, {'task_id': 5, 'passed': True, 'result': 'passed', 'completion_id': 0})], 7: [(0, {'task_id': 7, 'passed': True, 'result': 'passed', 'completion_id': 0})], 8: [(0, {'task_id': 8, 'passed': True, 'result': 'passed', 'completion_id': 0})], 9: [(0, {'task_id': 9, 'passed': True, 'result': 'passed', 'completion_id': 0})], 10: [(0, {'task_id': 10, 'passed': True, 'result': 'passed', 'completion_id': 0})], 11: [(0, {'task_id': 11, 'passed':